In [ ]:
import pandas as pd
import time 
import os
import copy
class bms_code:
    url = 'https://raw.githubusercontent.com/guru99-edu/R-Programming/master/adult_data.csv'
        
    def __init__(self):
        self.data = pd.read_csv(self.url)
        
    def na_check(self):
        # default NA list
        nalists = ['?', 'NA', 'na', 'null', 'Null', 'NULL', ' ']
        
        # asking that anything to add
        addlist = input(f'기본 설정된 결측값 리스트: {nalists} \n 추가하고 싶은 리스트를 작성해주십시오:')
        if len(addlist) != 0:
            nalists += addlist.split(sep=',')            
        
        # change all NA
        self.data[self.data.isin(nalists)] = np.nan
        
test = bms_code()

In [ ]:
test.na_check()

In [ ]:
"""
결측값 파악 및 정의

['?', 'NA', 'na', 'null', 'Null', 'NULL', ' '] 해당 문자들을 결측값으로 인식하며, 결측값 처리여부를 묻는다.
이부분은 정의 해주는게 좋을 거 같은데..?

"""
def na_check(df, na_list = ['?', 'NA', 'na', 'null', 'Null', 'NULL', ' ']):

    for c in candidate:
        answer = input(f"{c}를 결측값으로 처리할까요? y 또는 n을 입력하세요")

        if answer is 'y':
            # 결측값은 numpy의 nan을 활용한다.
            df = df.replace({c: np.nan})        
        else:
            print('y또는 n값만 허용합니다')
            break
    
    return(df)

In [ ]:
"""
데이터 품질 확인

"""
def dqc(df):
    # 결과를 dict로 저장한다.
    output = dict()
    
    rows = df.shape[0] 
    cols = df.shape[1]
    
    # data 내 전체 결측값을 확인한다.
    total_null = sum(df.isnull().sum())
    
    # 중복되는 row가 있는지 확인한다. 
    duplicate_row = sum(df.duplicated())
    
    # 중복 row의 index를 확인한다.
    duplicate_index = [idx for idx, result in df.duplicated().to_dict().items() if result is True]

    output['dataset'] = {
        'rows': rows,
        'cols': cols,
        'null': total_null,
        'duplicate row': duplicate_row,
        'duplicate row index': duplicate_index
    }

    column_names = df.columns.to_list()
    
    # 깂 전체가 결측값인 column을 확인한다.
    null_count_dict = df.isnull().sum().to_dict()
    all_null_column = [column for column, value in null_count_dict.items() if value == rows]
    
    # 값 전체가 동일한 column을 확인한다.
    all_same_column_dict = np.var(df).to_dict()
    all_same_column = [column for column, var in all_same_column_dict.items() if var == 0]
    
    # 모든값이 중복되는 column을 확인한다.
    duplicate_column_dict = df.transpose().duplicated().to_dict()
    duplicate_column = [column for column, result in duplicate_column_dict.items() if result is True]

    variables = dict()
    string_variable = dict()
    numeric_variable = dict()

    for column_name in column_names:

        temp_dict = dict()
        
        # 값 전체가 결측값인 column은 all_null 값이 1로 입력된다. 
        if column_name in all_null_column:
            temp_dict['all_null'] = 1 
        else: 
            temp_dict['all_null'] = 0

        # column 내에 결측값 갯수를 확인한다.
        temp_dict['null_count'] = null_count_dict[column_name] 
        
        # column 내에 unique 값 갯수를 확인한다.
        unique_value = list(df[column_name].unique())
        unique_count = len(unique_value)
        temp_dict['unique_count'] = unique_count
        
        # 값 전체가 동일한 column은 all_same 값이 1로 입력된다.
        if unique_count == 1:
            temp_dict['all_same'] = 1 
        else: 
            temp_dict['all_same'] = 0
        
        # 중복되는 column일 경우 duplicate 값이 1로 입력된다.
        if column_name in duplicate_column:
            temp_dict['duplicate'] = 1 
        else: 
            temp_dict['duplicate'] = 0

        # is_numeric_dtype 함수를 가지고 column의 string, numeric type을 구분한다.     
        if is_numeric_dtype(df[column_name]) is True:
            temp_dict['min'] = min(df[column_name])
            temp_dict['avg'] = round(sum(df[column_name]) / len(df[column_name]), 2)
            temp_dict['max'] = max(df[column_name])

            numeric_variable[column_name] = temp_dict
        else: 
            values = df[column_name].astype(str)
            values_percent = round(values.groupby(values).count() / len(values), 2)
            temp_dict['values_percent'] = values_percent.to_dict()

            string_variable[column_name] = temp_dict

    output['variable'] = {
        'numeric_variable': numeric_variable,  
        'string_variable': string_variable      
    }

    return(output)

In [ ]:
na_check_df = na_check(income_data)

output = dqc(na_check_df)

In [ ]:
# 결과를 json으로 저장한다.
json.dump(output, open("output.json", 'w'))